In [ ]:
import json
import os
import math
import librosa
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv1D, BatchNormalization, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

In [ ]:
import json
import os

# Spécifiez le chemin du dossier dans lequel vous souhaitez créer le fichier
dossier = "C:/Users/asus/Desktop/rnn_nounamo"

# Assurez-vous que le dossier existe, sinon, créez-le
if not os.path.exists(dossier):
    os.makedirs(dossier)


# Construisez le chemin complet du fichier data.json
chemin_fichier = os.path.join(dossier, "data_5.json")

# Écrivez les données dans le fichier JSON
with open(chemin_fichier, 'w') as fichier_json:
    json.dump({}, fichier_json, indent=4)

print(f'Le fichier data.json a été créé dans le dossier : {dossier}')


In [ ]:
import json
import os
import math
import librosa
import warnings

# Ignorer les avertissements FutureWarning spécifiques à librosa
warnings.filterwarnings("ignore", category=FutureWarning, module="librosa")

DATASET_PATH = "C:/Users/asus/Desktop/rnn_nounamo/africa_songs"
JSON_PATH = "C:/Users/asus/Desktop/rnn_nounamo/data.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION


def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    """Extracts MFCCs from music dataset and saves them into a json file along witgh genre labels.

        :param dataset_path (str): Path to dataset
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # loop through all genre sub-folder
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # ensure we're processing a genre sub-folder level
        if dirpath is not dataset_path:

            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            # process all audio files in genre sub-dir
            for f in filenames:

# load audio file
                file_path = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # process all segments of audio file
                for d in range(num_segments):

                    # calculate start and finish sample for current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # extract mfcc
                    mfcc = librosa.feature.mfcc(y=signal[start:finish],sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T

                    # store only mfcc feature with expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path, d+1))

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)
        
        
if __name__ == "__main__":
    save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv1D, BatchNormalization, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

# Chemin vers le fichier JSON contenant les données
DATA_PATH = "C:/Users/asus/Desktop/rnn_nounamo/data.json"

# Fonction pour charger les données à partir du fichier JSON
def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

# Fonction pour afficher les courbes d'entraînement
def plot_history(history):
    fig, axs = plt.subplots(2)

    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="validation accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy Evaluation")

    axs[1].plot(history.history["loss"], label="train loss")
    axs[1].plot(history.history["val_loss"], label="validation loss")
    axs[1].set_ylabel("Loss")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Loss Evaluation")

    plt.show()

# Fonction pour préparer les ensembles de données d'entraînement, de validation et de test
def prepare_datasets(test_size, validation_size):
    X, y = load_data(DATA_PATH)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_validation, y_train, y_validation = train_test_split(
        X_train, y_train, test_size=validation_size, random_state=42
    )

    return X_train, X_validation, X_test, y_train, y_validation, y_test

# Fonction pour construire le modèle
def build_model(input_shape):
    model = keras.Sequential()
    
    model.add(Conv1D(64, 5, strides=2, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())

    model.add(GRU(128, return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.01)))
    model.add(GRU(128, kernel_regularizer=keras.regularizers.l2(0.01)))
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(10, activation='softmax'))

    return model

# Partie principale du script
if __name__ == "__main__":
    # Chargement des données et préparation des ensembles
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    # Définition de la forme d'entrée du modèle
    input_shape = (X_train.shape[1], X_train.shape[2])

    # Construction du modèle
    model = build_model(input_shape)

    # Compilation du modèle avec l'optimiseur, la fonction de perte et les métriques
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Affichage de la structure du modèle
    model.summary()

    # Définition des callbacks pour l'arrêt précoce et la sauvegarde du meilleur modèle
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

    # Entraînement du modèle
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_validation, y_validation),
        batch_size=32,
        epochs=100,
        callbacks=[early_stopping, checkpoint],
        verbose=1,
    )

    # Affichage des courbes d'entraînement
    plot_history(history)

    # Évaluation du modèle sur l'ensemble de test
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nPrécision sur le test:', test_acc)
